---

# import &hàm khác

In [ ]:
# !git clone https://github.com/thanhtruong1712/DAMH_LTSSUD.git

In [ ]:
#import

import numpy as np
import cv2 as cv
import numba as nb
from numba import jit, njit, cuda,  uint32, uint64, float32, float64
from numpy import array, empty_like, empty, zeros
from math import exp
from queue import Queue
import json
import math

BLOCK_SIZE_2 = (32, 32)
width=1365
height=2048
# width*height
grid_size_2 = (math.ceil(width / BLOCK_SIZE_2[0]),math.ceil(height / BLOCK_SIZE_2[1]))

BLOCK_SIZE_1=32
grid_size_1 = math.ceil(width*height/ BLOCK_SIZE_1)

In [ ]:
def applyCannyThreshold( frame, val):
    ratio = 1.2
    kernel_size = 3
    low_threshold = val
    img_blur = cv.GaussianBlur(frame, (3, 3), 0)
    detected_edges = cv.Canny(img_blur, low_threshold, low_threshold*ratio, kernel_size)
    mask = detected_edges != 0
    dst = frame * (mask[:,:].astype(frame.dtype))
    return dst

In [ ]:
def zipImage(src, zip_x, zip_y, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    
    for idx in range(0, zip_rs * zip_y, zip_y):
        for jdx in range(0, zip_cs * zip_x, zip_x):
            block_img = src[idx : idx + zip_y, jdx : jdx + zip_x]
            num_pixel = np.sum(block_img > 0)
            if num_pixel >= zip_x*zip_y * ratio:
                block_img[:,:] = 1
            else:
                block_img[:,:] = 0

    return src

In [ ]:
# Hàm dùng để liên kết các ô xung quanh để lấp khuyết sẽ trả ra kết quả là một ma trận mask
def joinNeiboorPixel(src, zip_x, zip_y, mask_size, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    half = int(mask_size / 2)
    dst = src.copy()
    for idx in range(half , zip_rs - half):
        for jdx in range(half, zip_cs - half):
            start_row_mask = (idx - half ) * zip_y
            end_row_mask = (idx + half + 1) * zip_y
            start_col_mask = (jdx - half) * zip_x
            end_col_mask = (jdx + half + 1) * zip_x
            mask_block = src[start_row_mask : end_row_mask, start_col_mask : end_col_mask]
            block_img = dst[idx * zip_y : (idx + 1) * zip_y, jdx * zip_x : (jdx + 1) * zip_x]
            num_pixel = np.sum(mask_block > 0)
            if num_pixel >= mask_size * mask_size * zip_x * zip_y * ratio:
                block_img[:,:] = 1
    return dst

# 2. Cài đặt tuần tự

In [ ]:
def applyCannyThreshold( frame, val):
    ratio = 1.2
    kernel_size = 3
    low_threshold = val
    img_blur = cv.GaussianBlur(frame, (3, 3), 0)
    detected_edges = cv.Canny(img_blur, low_threshold, low_threshold*ratio, kernel_size)
    mask = detected_edges != 0
    dst = frame * (mask[:,:].astype(frame.dtype))
    return dst

In [ ]:
# Hàm dùng để nén ảnh với threshold là ratio sẽ trả ra kết quả là ảnh với block tương ứng với zip x zip
# và nếu block nào không đủ pixel yêu cầu thì là 0 còn ngược lại thì là 1
@jit
def zipImage(src, zip_x, zip_y, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    for idx in range(0, zip_rs * zip_y, zip_y):
        for jdx in range(0, zip_cs * zip_x, zip_x):
            block_img = src[idx : idx + zip_y, jdx : jdx + zip_x]
            num_pixel = np.sum(block_img > 0)
            if num_pixel >= zip_x*zip_y * ratio:
                block_img[:,:] = 1
            else:
                block_img[:,:] = 0

    return src

@jit
# Hàm dùng để liên kết các ô xung quanh để lấp khuyết sẽ trả ra kết quả là một ma trận mask
def joinNeiboorPixel(src, zip_x, zip_y, mask_size, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    half = int(mask_size / 2)
    dst = src.copy()
    for idx in range(half , zip_rs - half):
        for jdx in range(half, zip_cs - half):
            start_row_mask = (idx - half ) * zip_y
            end_row_mask = (idx + half + 1) * zip_y
            start_col_mask = (jdx - half) * zip_x
            end_col_mask = (jdx + half + 1) * zip_x
            mask_block = src[start_row_mask : end_row_mask, start_col_mask : end_col_mask]
            block_img = dst[idx * zip_y : (idx + 1) * zip_y, jdx * zip_x : (jdx + 1) * zip_x]
            num_pixel = np.sum(mask_block > 0)
            if num_pixel >= mask_size * mask_size * zip_x * zip_y * ratio:
                block_img[:,:] = 1
    return dst


@jit
def convert_rgb2gray(in_pixels, out_pixels):
    '''
    Convert color image to grayscale image.
 
    in_pixels : numpy.ndarray with shape=(h, w, 3)
                h, w is height, width of image
                3 is colors with BGR (blue, green, red) order
        Input RGB image
    
    out_pixels : numpy.ndarray with shape=(h, w)
        Output image in grayscale
    '''
    for r in range(len(in_pixels)):
        for c in range(len(in_pixels[0])):
            out_pixels[r, c] = (in_pixels[r, c, 0] * 0.114 + 
                                in_pixels[r, c, 1] * 0.587 + 
                                in_pixels[r, c, 2] * 0.299)
    return out_pixels

@jit
def cvMoments(img,m_,mu_,nu_):
    '''
    input:  ảnh 2D
        m_ mảng 3x3 : M ~ raw Moments
        mu_ 3x3: mu ~ central moments
        nu_ 3x3: nu ~ normalized central moments 
    output: nu_
    '''
    # tính m_
    for i in range(4):
        for j in range(4):
            temp=0
            if (i,j) not in [(1,3),(2,2),(2,3)]:
                for x in range(img.shape[0]):
                    for y in range(img.shape[1]):
                        temp=temp+img[x,y]*(x**i)*(y**j)
                m_[i,j]=temp
            if i==3:
                break
    # xbar ybar
    xbar=m_[1,0]/m_[0,0]
    ybar=m_[0,1]/m_[0,0]
    # tính mu
    mu_[1,1] = m_[1,1] - xbar*m_[0,1]
    mu_[0,2] = m_[2,0] - xbar*m_[1,0]
    mu_[2,0] = m_[0,2] - ybar*m_[0,1]
    mu_[1,2] = m_[2,1] - 2*xbar*m_[1,1] - ybar*m_[2,0] + 2*(xbar**2)*m_[0,1]
    mu_[2,1] = m_[1,2] - 2*ybar*m_[1,1] - xbar*m_[0,2] + 2*(ybar**2)*m_[1,0]
    mu_[0,3] = m_[3,0] - 3*xbar*m_[2,0] + 2*(xbar**2)*m_[1,0]
    mu_[3,0] = m_[0,3] - 3*ybar*m_[0,2] + 2*(ybar**2)*m_[0,1]

    #tính nu  nu_ji = mu_ji / [m00^(((i+j)/2)+1)]
    for i in range(4):
        for j in range(4):
            nu_[i,j] = mu_[i,j]/(m_[0,0]**(((i+j)/2)+1))
 
    return nu_

# HuMoments
@jit
def cvHuMoments(eta,hu_):
    '''
    eta: output của cvMoments
    hu_: Mảng 1D 7 giá trị 
    '''
    hu_[0] =  eta[2][0] + eta[0][2]
    hu_[1] = (eta[2][0] - eta[0][2])**2 + 4*eta[1][1]**2
    hu_[2] = (eta[3][0] - 3*eta[1][2])**2 + (3*eta[2][1] - eta[0][3])**2
    hu_[3] = (eta[3][0] + eta[1][2])**2 + (eta[2][1] + eta[0][3])**2
    hu_[4] = (eta[3][0] - 3*eta[1][2])*(eta[3][0] + eta[1][2])*((eta[3][0] + eta[1][2])**2 - 3*(eta[2][1] + eta[0][3])**2) +\
        (3*eta[2][1] - eta[0][3])*(eta[2][1] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2 - (eta[2][1] + eta[0][3])**2)
    hu_[5] = (eta[2][0] - eta[0][2])*((eta[3][0] + eta[1][2])**2 - (eta[2][1] + eta[0][3])**2) + \
          4*eta[1][1]*(eta[3][0] + eta[1][2])*(eta[2][1] + eta[0][3])
    hu_[6] = (3*eta[2][1] - eta[0][3])*(eta[2][1] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2-(eta[2][1] + eta[0][3])**2) -\
          (eta[3][0] - 3*eta[1][2])*(eta[1][2] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2-(eta[2][1] + eta[0][3])**2)
    return hu_


@jit
def compute_hist(img, hist):
    '''
    Color histogram: thống kê số lần xuất hiện các mức sáng trong ảnh với bins=8, 
                     phạm vi [0,256] cho mỗi kênh màu
    input: 
            img: numpy.ndarray with shape=(h, w, 3)
            hist: numpy.ndarray with shape=(8,8,8)
    '''
    h, w,d = img.shape[:3] 
    for i in range(h): 
        for j in range(w): 
            x,y,z=img[i][j][0]//32,img[i][j][1]//32,img[i][j][2]//32
            hist[x][y][z] =hist[x][y][z] + 1 
    return hist


In [ ]:
# #huMoments
# hu=np.zeros(7,float) 
# hu_moments = cvHuMoments(image, hu)


In [ ]:
def fd_histogram2(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV) 
    # compute the color histogram
    hist = np.zeros((8,8,8), np.float32) 
    hist  = compute_hist(image,hist)
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def getFigureForImage2(path):
    img = cv.imread(path)
    # gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    gray = np.empty((height, width), dtype=img.dtype)
    gray = convert_rgb2gray(img,gray)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img
    # print(img.shape)
    hist_figure = fd_histogram2(img).astype(np.float64)
    #huMoments
    m=np.zeros((4, 4), dtype=np.float64)
    mu=np.zeros((4, 4), dtype=np.float64)
    nu=np.zeros((4, 4), dtype=np.float64)
    # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    gray_img = np.empty((height, width), dtype=img.dtype)
    img=convert_rgb2gray(img,gray_img)
    nu2=cvMoments(img,m,mu,nu)
    hu=np.zeros(7,float) 
    hu_moments = cvHuMoments(nu2, hu)
    fig = np.concatenate((hist_figure, hu_moments))
    return fig
y=getFigureForImage2("/content/Test_4.jpg")
# tuần tự

In [ ]:

#không tuần tự
def fd_histogram(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
# fd_hu_moments(getImageToTest())    


def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
y2=getFigureForImage("/content/Test_4.jpg")
np.mean(np.abs(y-y2))

0.0002675324963013007

In [ ]:
# nháp


In [ ]:
%%time
y=getFigureForImage2("/content/Test_4.jpg")
# CPU times: user 284 ms, sys: 420 µs, total: 285 ms
# Wall time: 265 ms

CPU times: user 265 ms, sys: 696 µs, total: 266 ms
Wall time: 235 ms


In [ ]:
%%time
y2=getFigureForImage("/content/Test_4.jpg")

CPU times: user 177 ms, sys: 3.02 ms, total: 180 ms
Wall time: 143 ms


---

# 3. Cài đặt song song

In [ ]:
!pip install -U xgboost==1.5.1 numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173.5 MB 10.0 kB/s 
     |████████████████████████████████| 3.3 MB 13.5 MB/s 
     |████████████████████████████████| 34.5 MB 1.2 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
!python /content/full_test.py /content/Test_4.jpg \
/content/xgb_hyperparams.json /content/xgb_tree.json

python3: can't open file '/content/full_test.py': [Errno 2] No such file or directory


### gray

In [ ]:

import cv2 as cv
import numpy as np
from numba import cuda, jit, njit
@jit
def convert_rgb2gray(in_pixels, out_pixels):
    '''
    Convert color image to grayscale image.
 
    in_pixels : numpy.ndarray with shape=(h, w, 3)
                h, w is height, width of image
                3 is colors with BGR (blue, green, red) order
        Input RGB image
    
    out_pixels : numpy.ndarray with shape=(h, w)
        Output image in grayscale
    '''
    for r in range(len(in_pixels)):
        for c in range(len(in_pixels[0])):
            out_pixels[r, c] = (in_pixels[r, c, 0] * 0.114 +  #r*0.114 + g*0.587 + b*0.299
                                in_pixels[r, c, 1] * 0.587 + 
                                in_pixels[r, c, 2] * 0.299)
    return out_pixels
img = cv.imread("/content/Test_4.jpg")
height, width = img.shape[:2]
gray_img = np.empty((height, width), dtype=img.dtype)
gray1=convert_rgb2gray(img,gray_img)

In [ ]:
#ok
@cuda.jit
def convert_rgb2gray_kernel(in_pixels, out_pixels):
    c, r = cuda.grid(2)
    if r < out_pixels.shape[0] and c < out_pixels.shape[1]:
        out_pixels[r, c] = (in_pixels[r, c, 0] * 0.114 + 
                            in_pixels[r, c, 1] * 0.587 + 
                            in_pixels[r, c, 2] * 0.299)


In [ ]:
# test
import math
BLOCK_SIZE = (32, 32)
grid_size = (math.ceil(width / BLOCK_SIZE[0]),math.ceil(height / BLOCK_SIZE[1]))
d_in_img = cuda.to_device(img)
d_gray_img = cuda.device_array((height, width), dtype=np.uint8)
convert_rgb2gray_kernel[grid_size, BLOCK_SIZE](d_in_img, d_gray_img)


In [ ]:
gray2=d_gray_img.copy_to_host()
# gray2

In [ ]:
np.mean(np.abs(gray2.astype(np.int16)- gray1))


0.0009211166437728938

In [ ]:
# cài đặt song song hóa
import numba as nb
@cuda.jit(nb.void(nb.uint8[:, :, ::1], nb.uint8[:, ::1]))
def convert_rgb2gray_kernel(in_pixels, out_pixels):
    c, r = cuda.grid(2)
    if r < out_pixels.shape[0] and c < out_pixels.shape[1]:
        out_pixels[r, c] = (in_pixels[r, c, 0] * 0.114 + 
                            in_pixels[r, c, 1] * 0.587 + 
                            in_pixels[r, c, 2] * 0.299)

BLOCK_SIZE = (32, 32)
def convert_rgb2gray_use_kernel(image):
    height, width = image.shape[:2]
    grid_size = (math.ceil(width / BLOCK_SIZE[0]),math.ceil(height / BLOCK_SIZE[1]))
    d_in_img = cuda.to_device(image)
    d_gray_img = cuda.device_array((height, width), dtype=np.uint8)
    start = timer()
    convert_rgb2gray_kernel[grid_size, BLOCK_SIZE](d_in_img, d_gray_img)
    end = timer()
    total_time = (end - start) * 1000
    print(f'Total time kernel: {total_time:0.6f} ms\n')
    
    gray=d_gray_img.copy_to_host()
    return gray

def fd_hu_moments3(image):
    image=convert_rgb2gray_use_kernel(image)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature

def getFigureForImage3(path):
    img = cv.imread(path)
    gray = convert_rgb2gray_use_kernel(img)
    gray = cv.GaussianBlur(gray, (3, 3), 0)

    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)

    for chanel in range(3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments3(img)

    fig = np.concatenate((hist_figure, hu_monents))
    return fig

# test song song hóa
from timeit import default_timer as timer
def compare_gray(path):
    img = cv.imread(path)
    start = timer()
    height, width = img.shape[:2]
    gray_img = np.empty((height, width), dtype=img.dtype)
    gray1= convert_rgb2gray(img,gray_img)
    end = timer()
    total_time = (end - start) * 1000
    print(f'Total time on host 1 : {total_time:0.6f} ms\n')
    
    start = timer()
    height, width = img.shape[:2]
    gray_img = np.empty((height, width), dtype=img.dtype)
    gray2= convert_rgb2gray_use_kernel(img)
    end = timer()
    total_time = (end - start) * 1000
    print(f'Total time 2: {total_time:0.6f} ms\n')
    print('Convert rgb to grayscale error:',np.mean(np.abs(gray2.astype(np.int16)- gray1)))


In [ ]:
compare_gray("/content/Test_4.jpg")

Total time on host 1 : 7.845192 ms

Total time kernel: 0.358635 ms

Total time 2: 4.991006 ms

Convert rgb to grayscale error: 0.0009211166437728938


### hist

In [ ]:
@jit
def compute_hist(img, hist):
    '''
    Color histogram: thống kê số lần xuất hiện các mức sáng trong ảnh với bins=8, 
                     phạm vi [0,255] cho mỗi kênh màu
    input: 
            img: numpy.ndarray with shape=(h, w, 3)
            hist: numpy.ndarray with shape=(8,8,8) 
    '''
    h, w,d = img.shape[:3] 
    for i in range(h): 
        for j in range(w): 
            x,y,z=img[i][j][0]//32,img[i][j][1]//32,img[i][j][2]//32
            hist[x][y][z] =hist[x][y][z] + 1 
    return hist

In [ ]:
@cuda.jit
def histogram_kernel(d_input,dhist):
    '''
    2d & 1d
    '''
    i = cuda.grid(1)
    if i < d_input.size:
        x,y,z=d_input[i][0]//32,d_input[i][1]//32,d_input[i][2]//32
        index=x*8*8+y*8+z
        cuda.atomic.add(dhist,index, 1)
        # cuda.syncthreads() 

temp=cv.imread("/content/Test_4.jpg")
hist21 = np.zeros((8,8,8), np.float32) 
hist21  = compute_hist(temp,hist21)

hist25 = np.zeros(8*8*8, np.float32) 
h, w,d = temp.shape[:3] 
temp=temp.reshape(w*h,3)
BLOCK_SIZE=32
grid_size = math.ceil(w*h/ BLOCK_SIZE)
histogram_kernel[grid_size, BLOCK_SIZE](temp,hist25)
np.mean(np.abs(hist25.flatten().astype(np.int32)- hist21.flatten()))       

0.0

In [ ]:
temp=cv.imread("/content/Test_4.jpg")
hist21 = np.zeros((8,8,8), np.float32) 

In [ ]:
%%time
hist21  = compute_hist(temp,hist21)

CPU times: user 8.02 ms, sys: 0 ns, total: 8.02 ms
Wall time: 7.98 ms


In [ ]:
hist25 = np.zeros(8*8*8, np.float32) 
temp=cv.imread("/content/Test_4.jpg")
h, w,d = temp.shape[:3] 
temp=temp.reshape(w*h,3)
BLOCK_SIZE=32
grid_size = math.ceil(w*h/ BLOCK_SIZE)

In [ ]:
%%time
histogram_kernel[grid_size, BLOCK_SIZE](temp,hist25)

CPU times: user 8.81 ms, sys: 1.75 ms, total: 10.6 ms
Wall time: 13.2 ms


In [ ]:
d_in_img = cuda.to_device(temp.reshape(w*h,3))
d_hist = cuda.device_array(512, dtype=np.float32)

# gray2=d_gray_img.copy_to_host()

In [ ]:
%%time
histogram_kernel[grid_size, BLOCK_SIZE](d_in_img, d_hist)

CPU times: user 1.27 ms, sys: 29 µs, total: 1.3 ms
Wall time: 1.05 ms


### hist3d

In [ ]:
@jit
def compute_hist(img, hist):
    '''
    Color histogram: thống kê số lần xuất hiện các mức sáng trong ảnh với bins=8, 
                     phạm vi [0,255] cho mỗi kênh màu
    input: 
            img: numpy.ndarray with shape=(h, w, 3)
            hist: numpy.ndarray with shape=(8,8,8) 
    '''
    h, w,d = img.shape[:3] 
    for i in range(h): 
        for j in range(w): 
            x,y,z=img[i][j][0]//32,img[i][j][1]//32,img[i][j][2]//32
            hist[x][y][z] =hist[x][y][z] + 1 
    return hist

In [ ]:
@cuda.jit
def histogram_kernel(d_input,dhist):
    '''
    2d & 3d
    '''
    i = cuda.grid(1)
    if i < d_input.size:
        x,y,z=d_input[i][0]//32,d_input[i][1]//32,d_input[i][2]//32
        # index=x*8*8+y*8+z
        cuda.atomic.add(dhist,(x,y,z), 1)
        cuda.syncthreads() 

temp=cv.imread("/content/Test_4.jpg")
hist21 = np.zeros((8,8,8), np.float32) 
hist21  = compute_hist(temp,hist21)

hist25 = np.zeros((8,8,8), np.float32) 
h, w,d = temp.shape[:3] 
temp=temp.reshape(w*h,3)
BLOCK_SIZE=32
grid_size = math.ceil(w*h/ BLOCK_SIZE)
histogram_kernel[grid_size, BLOCK_SIZE](temp,hist25)
np.mean(np.abs(hist25.flatten().astype(np.int32)- hist21.flatten()))       

0.0

### hist3d_img3d_ok

In [ ]:
@jit
def compute_hist(img, hist):
    '''
    Color histogram: thống kê số lần xuất hiện các mức sáng trong ảnh với bins=8, 
                     phạm vi [0,255] cho mỗi kênh màu
    input: 
            img: numpy.ndarray with shape=(h, w, 3)
            hist: numpy.ndarray with shape=(8,8,8) 
    '''
    h, w,d = img.shape[:3] 
    for i in range(h): 
        for j in range(w): 
            x,y,z=img[i][j][0]//32,img[i][j][1]//32,img[i][j][2]//32
            hist[x][y][z] =hist[x][y][z] + 1 
    return hist

In [ ]:
@cuda.jit
def histogram_kernel(d_input,dhist): # khởi tạo hist device, check 
    '''
    3d & 3d

    '''
    r,c = cuda.grid(2)
    if r < d_input.shape[0] and c < d_input.shape[1]:
        x,y,z=d_input[r][c][0]//32,d_input[r][c][1]//32,d_input[r][c][2]//32
        cuda.atomic.add(dhist,(x,y,z), 1)

temp=cv.imread("/content/Test_4.jpg")
hist21 = np.zeros((8,8,8), np.float32) 
hist21  = compute_hist(temp,hist21)

hist25 = np.zeros((8,8,8), np.float32) 
h, w,d = temp.shape[:3] 
histogram_kernel[grid_size_2, BLOCK_SIZE_2](temp,hist25)
np.mean(np.abs(hist25.flatten().astype(np.int32)- hist21.flatten()))       

0.0

### HuMoments

#### Moments

In [ ]:
def m_1(img,m_): # img 
    for i in range(4):
        for j in range(4):
            temp=0
            if (i,j) not in [(1,3),(2,2),(2,3)]:
                for x in range(img.shape[0]):
                    for y in range(img.shape[1]):
                        temp=temp+img[x,y]*(x**i)*(y**j)
                m_[i,j]=temp
            if i==3:
                break
    return m_
m1_jit=jit(m_1)

@cuda.jit
def m_1_kernel(img,m_): # img 
    r = cuda.grid(1)
    if r < width*height:
        for i in range(4):
            for j in range(4):
                if (i,j) not in [(1,3),(2,2),(2,3)]:
                    temp_ij=img[r]*((r//height)**i)*((r%height)**j)
                    cuda.atomic.add(m_, (i,j),temp_ij)              
                if i==3:    
                    break


In [ ]:
image = cv.cvtColor(cv.imread("/content/Test_4.jpg"), cv.COLOR_BGR2GRAY)
m1=np.zeros((4, 4), dtype=np.float64)
m1=m1_jit(image,m1)
m2=np.zeros((4, 4), dtype=np.float64)
m_1_kernel[grid_size_1, BLOCK_SIZE_1](image.flatten(),m2)
np.mean(np.abs(m2.astype(np.float32)- m1))
image[0,0]

TypingError: ignored

In [ ]:
m1 #136 ms

In [ ]:
m2 #64 ms

#### cv


In [ ]:
@jit
def cvMoments3(img,m_,mu_,nu_):
    # tính m_
    # xbar ybar
    xbar=m_[1,0]/m_[0,0]
    ybar=m_[0,1]/m_[0,0]
    # tính mu
    mu_[1,1] = m_[1,1] - xbar*m_[0,1]
    mu_[0,2] = m_[2,0] - xbar*m_[1,0]
    mu_[2,0] = m_[0,2] - ybar*m_[0,1]
    mu_[1,2] = m_[2,1] - 2*xbar*m_[1,1] - ybar*m_[2,0] + 2*(xbar**2)*m_[0,1]
    mu_[2,1] = m_[1,2] - 2*ybar*m_[1,1] - xbar*m_[0,2] + 2*(ybar**2)*m_[1,0]
    mu_[0,3] = m_[3,0] - 3*xbar*m_[2,0] + 2*(xbar**2)*m_[1,0]
    mu_[3,0] = m_[0,3] - 3*ybar*m_[0,2] + 2*(ybar**2)*m_[0,1]

    #tính nu  nu_ji = mu_ji / [m00^(((i+j)/2)+1)]
    for i in range(4):
        for j in range(4):
            nu_[i,j] = mu_[i,j]/(m_[0,0]**(((i+j)/2)+1))
 
    return nu_


In [ ]:
image = cv.cvtColor(cv.imread("/content/Test_4.jpg"), cv.COLOR_BGR2GRAY)
m1=np.zeros((4, 4), dtype=np.float64)
m1=m1_jit(image,m1)

In [ ]:
# m=np.zeros((4, 4), dtype=np.float64)
mu=np.zeros((4, 4), dtype=np.float64)
nu=np.zeros((4, 4), dtype=np.float64)
nu=cvMoments3(image,m1,mu,nu)

In [ ]:
nu

array([[ 0.00000000e+00,  0.00000000e+00,  5.16443078e-04,
         8.96370142e-07],
       [ 0.00000000e+00,  3.04189063e-05, -1.71052571e-06,
         0.00000000e+00],
       [ 1.03453212e-03,  2.27965685e-07,  0.00000000e+00,
         0.00000000e+00],
       [-5.33108232e-06,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [ ]:
hu=np.zeros(7,float) 
hu1=cvHuMoments(nu,hu)

### mo2

In [ ]:
def m_1(img,m_): # img 
    for i in range(4):
        for j in range(4):
            temp=0
            if (i,j) not in [(1,3),(2,2),(2,3)]:
                for x in range(img.shape[0]):
                    for y in range(img.shape[1]):
                        temp=temp+img[x,y]*(x**i)*(y**j)
                m_[i,j]=temp
            if i==3:
                break
    return m_
m1_jit=jit(m_1)

@cuda.jit
def m_ij_k(img,m_,i,j): # img 
    temp_ij=0
    r ,c= cuda.grid(2)
    if r < img.shape[0] and c < img.shape[1]:
        temp_ij=img[r,c]*(r**i)*(c**j)
        cuda.atomic.add(m_, (i,j),temp_ij)
        cuda.syncthreads()
        
def m_1_kernel(img,m_): # img 
    for i in range(4):
        for j in range(4):
            if (i,j) not in [(1,3),(2,2),(2,3)]:      
                m_ij_k[grid_size_2, BLOCK_SIZE_2](img,m_,i,j)      
            if i==3:    
                break

image = cv.cvtColor(cv.imread("/content/Test_4.jpg"), cv.COLOR_BGR2GRAY)#.astype(np.float32)

# host
m1=np.zeros((4, 4), dtype=np.float64)
m1=m1_jit(image,m1)

#kernel
d_in_img = cuda.to_device(image)
m2=np.zeros((4, 4), dtype=np.float64)
d_m2 = cuda.to_device(m2)
m_1_kernel(d_in_img,d_m2)
m2=d_m2.copy_to_host()
np.mean(np.abs(m2.astype(np.float32)- m1))
# image[0,0]

3070189894.25

In [ ]:
m1

array([[3.44355115e+08, 3.73548604e+11, 5.27892316e+14, 8.27065389e+17],
       [2.35280119e+11, 2.58833614e+14, 3.69009044e+17, 0.00000000e+00],
       [2.21994841e+14, 2.41980065e+17, 0.00000000e+00, 0.00000000e+00],
       [2.37334430e+17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
m2

array([[3.44355115e+08, 3.73548604e+11, 5.27892316e+14, 8.27065389e+17],
       [2.35280119e+11, 2.58833614e+14, 3.69009044e+17, 0.00000000e+00],
       [2.21994841e+14, 2.41980065e+17, 0.00000000e+00, 0.00000000e+00],
       [2.37334430e+17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
d_in_img = cuda.to_device(image)
m2=np.zeros((4, 4), dtype=np.float64)
d_m2 = cuda.to_device(m2)

In [ ]:
%%time
m_1_kernel(d_in_img,d_m2)
# m2=d_m2.copy_to_host()
# np.mean(np.abs(m2.astype(np.float32)- m1))

CPU times: user 3.04 ms, sys: 1.93 ms, total: 4.97 ms
Wall time: 5.66 ms


### mo2

In [ ]:

def m_1(img,m_): # img 
    for i in range(4):
        for j in range(4):
            temp=0
            if (i,j) not in [(1,3),(2,2),(2,3)]:
                for x in range(img.shape[0]):
                    for y in range(img.shape[1]):
                        temp=temp+img[x,y]*(x**i)*(y**j)
                m_[i,j]=temp
            if i==3:
                break
    return m_
m1_jit=jit(m_1)

@cuda.jit
def m_ij_k(img,m_,i,j): # img 
    temp_ij=0
    r ,c= cuda.grid(2)
    if r < img.shape[0] and c < img.shape[1]:
        temp_ij=img[r,c]*(r**i)*(c**j)
        cuda.atomic.add(m_, (i,j),temp_ij)
        cuda.syncthreads()
        
def m_1_kernel(img,m_): # img 
    for i in range(4):
        for j in range(4):
            if (i,j) not in [(1,3),(2,2),(2,3)]:      
                m_ij_k[grid_size_2, BLOCK_SIZE_2](img,m_,i,j)      
            if i==3:    
                break

image = cv.cvtColor(cv.imread("/content/Test_4.jpg"), cv.COLOR_BGR2GRAY)#.astype(np.float32)

# host
m1=np.zeros((4, 4), dtype=np.float64)
m1=m1_jit(image,m1)

#kernel
d_in_img = cuda.to_device(image)
m2=np.zeros((4, 4), dtype=np.float64)
d_m2 = cuda.to_device(m2)
m_1_kernel(d_in_img,d_m2)
m2=d_m2.copy_to_host()
np.mean(np.abs(m2.astype(np.float32)- m1))
# image[0,0]

3070189894.25

In [ ]:
m1

array([[3.44355115e+08, 3.73548604e+11, 5.27892316e+14, 8.27065389e+17],
       [2.35280119e+11, 2.58833614e+14, 3.69009044e+17, 0.00000000e+00],
       [2.21994841e+14, 2.41980065e+17, 0.00000000e+00, 0.00000000e+00],
       [2.37334430e+17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
m2

array([[3.44355115e+08, 3.73548604e+11, 5.27892316e+14, 8.27065389e+17],
       [2.35280119e+11, 2.58833614e+14, 3.69009044e+17, 0.00000000e+00],
       [2.21994841e+14, 2.41980065e+17, 0.00000000e+00, 0.00000000e+00],
       [2.37334430e+17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
d_in_img = cuda.to_device(image)
m2=np.zeros((4, 4), dtype=np.float64)
d_m2 = cuda.to_device(m2)

In [ ]:
%%time
m_1_kernel(d_in_img,d_m2)
# m2=d_m2.copy_to_host()
# np.mean(np.abs(m2.astype(np.float32)- m1))

CPU times: user 6.08 ms, sys: 1.03 ms, total: 7.1 ms
Wall time: 7.29 ms


### ghép ss hu

In [ ]:
def cvHuMoments_kernel(d_in_img,m_,mu_,nu_,hu_):
    # tính m_
    for i in range(4):
        for j in range(4):
            if (i,j) not in [(1,3),(2,2),(2,3)]:      
                m_ij_k[grid_size_2, BLOCK_SIZE_2](img,m_,i,j)      
        if i==3:    
            break

    # xbar ybar
    xbar=m_[1,0]/m_[0,0]
    ybar=m_[0,1]/m_[0,0]
    # tính mu
    mu_[1,1] = m_[1,1] - xbar*m_[0,1]
    mu_[0,2] = m_[2,0] - xbar*m_[1,0]
    mu_[2,0] = m_[0,2] - ybar*m_[0,1]
    mu_[1,2] = m_[2,1] - 2*xbar*m_[1,1] - ybar*m_[2,0] + 2*(xbar**2)*m_[0,1]
    mu_[2,1] = m_[1,2] - 2*ybar*m_[1,1] - xbar*m_[0,2] + 2*(ybar**2)*m_[1,0]
    mu_[0,3] = m_[3,0] - 3*xbar*m_[2,0] + 2*(xbar**2)*m_[1,0]
    mu_[3,0] = m_[0,3] - 3*ybar*m_[0,2] + 2*(ybar**2)*m_[0,1]

    #tính nu  nu_ji = mu_ji / [m00^(((i+j)/2)+1)]
    for i in range(4):
        for j in range(4):
            nu_[i,j] = mu_[i,j]/(m_[0,0]**(((i+j)/2)+1))
 
    hu_[0] =  nu_[2][0] + nu_[0][2]
    hu_[1] = (nu_[2][0] - nu_[0][2])**2 + 4*nu_[1][1]**2
    hu_[2] = (nu_[3][0] - 3*nu_[1][2])**2 + (3*nu_[2][1] - nu_[0][3])**2
    hu_[3] = (nu_[3][0] + nu_[1][2])**2 + (nu_[2][1] + nu_[0][3])**2
    hu_[4] = (nu_[3][0] - 3*nu_[1][2])*(nu_[3][0] + nu_[1][2])*((nu_[3][0] + nu_[1][2])**2 - 3*(nu_[2][1] + nu_[0][3])**2) +\
        (3*nu_[2][1] - nu_[0][3])*(nu_[2][1] + nu_[0][3])*(3*(nu_[3][0] + nu_[1][2])**2 - (nu_[2][1] + nu_[0][3])**2)
    hu_[5] = (nu_[2][0] - nu_[0][2])*((nu_[3][0] + nu_[1][2])**2 - (nu_[2][1] + nu_[0][3])**2) + \
          4*nu_[1][1]*(nu_[3][0] + nu_[1][2])*(nu_[2][1] + nu_[0][3])
    hu_[6] = (3*nu_[2][1] - nu_[0][3])*(nu_[2][1] + nu_[0][3])*(3*(nu_[3][0] + nu_[1][2])**2-(nu_[2][1] + nu_[0][3])**2) -\
          (nu_[3][0] - 3*nu_[1][2])*(nu_[1][2] + nu_[0][3])*(3*(nu_[3][0] + nu_[1][2])**2-(nu_[2][1] + nu_[0][3])**2)

image = cv.cvtColor(cv.imread("/content/Test_4.jpg"), cv.COLOR_BGR2GRAY)
m=np.zeros((4, 4), dtype=np.float64)
mu=np.zeros((4, 4), dtype=np.float64)
nu=np.zeros((4, 4), dtype=np.float64)
hu=np.zeros(7,np.float64) 

#kernel
d_in_img = cuda.to_device(image)
d_m = cuda.to_device(m)
d_mu = cuda.to_device(mu)
d_nu = cuda.to_device(nu)
d_hu = cuda.to_device(hu)

cvHuMoments_kernel(d_in_img,d_m,d_mu,d_nu,d_hu)

hu_ss=d_hu.copy_to_host()

# tt
m=np.zeros((4, 4), dtype=np.float64)
mu=np.zeros((4, 4), dtype=np.float64)
nu=np.zeros((4, 4), dtype=np.float64)
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
nu2=cvMoments(img,m,mu,nu)
hu=np.zeros(7,float) 
hu_moments = cvHuMoments(nu2, hu)

np.mean(np.abs(hu_moments.astype(np.float32)- hu_ss.astype(np.float32)))

TypingError: ignored

### hsv k jit
image = cv.cvtColor(image, cv.COLOR_BGR2HSV) 

In [ ]:
#k jit được
# l@jit
def convert_bgr2hsv(in_pixels, out_pixels):
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0],in_pixels[i,j,1],in_pixels[i,j,2]
            S=0
            H=0
            V = max(r,g,b)
            # Vsmin=V-min(r,g,b)
            if V!=0:
                S=(V-min(r,g,b))/V # xét nguyên hay k
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            out_pixels[i,j] = (H,S,V)
            # out_pixels[i,j,0] = H
            # out_pixels[i,j,1] = S
            # out_pixels[i,j,2] = V
    return out_pixels

img=cv.imread("/content/Test_4.jpg").astype(np.float32)
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread("/content/Test_4.jpg").astype(np.float32)
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)
np.mean(np.abs(c1.astype(np.float32)- c2.astype(np.float32)))

<ipython-input-20-b83fd2d106fa>:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "convert_bgr2hsv" failed type inference due to: No implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(float32, 3d, C), UniTuple(int64 x 2), Tuple(float64, float64, float32))
 
There are 16 candidate implementations:
  - Of which 16 did not match due to:
  Overload of function 'setitem': File: <numerous>: Line N/A.
    With argument(s): '(array(float32, 3d, C), UniTuple(int64 x 2), Tuple(float64, float64, float32))':
   No match.

During: typing of setitem at <ipython-input-20-b83fd2d106fa> (23)

File "<ipython-input-20-b83fd2d106fa>", line 23:
def convert_bgr2hsv(in_pixels, out_pixels):
    <source elided>
                H = H + 360
            out_pixels[i,j] = (H,S,V)
            ^

  @jit
<ipython-input-20-b83fd2d106fa>:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT loopli

KeyboardInterrupt: ignored

In [ ]:
c1[0,2]

array([ 63.620693 ,   0.5550239, 209.       ], dtype=float32)

In [ ]:
c1[0,2]

array([ 32, 142, 209], dtype=uint8)

In [ ]:
c2[0,0]

array([196.03448  ,   0.5550239, 209.       ], dtype=float32)

In [ ]:
img=cv.imread("/content/Test_4.jpg").astype(np.float32)
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)

In [ ]:
@jit
def t(a,b,c):
    return max(a,b,c)
i=0
j=0
b,g,r = img[i,j,0],img[i,j,1],img[i,j,2]
t(b,g,r)

209

### hsv
image = cv.cvtColor(image, cv.COLOR_BGR2HSV) 

In [ ]:
@jit
def convert_bgr2hsv(in_pixels, out_pixels):
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0],in_pixels[i,j,1],in_pixels[i,j,2]
            S=0
            H=0
            V = max(r,g,b)
            if V!=0:
                S=(V-min(r,g,b))/V 
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            out_pixels[i,j,0] = H
            out_pixels[i,j,1] = S
            out_pixels[i,j,2] = V
    return out_pixels

img=cv.imread("/content/Test_4.jpg").astype(np.float32)
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread("/content/Test_4.jpg").astype(np.float32)
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)
np.mean(np.abs(c1.astype(np.float32)- c2.astype(np.float32)))

3.6882642e-07

In [ ]:
c2[0,3]

array([ 63.62069  ,   0.5550239, 209.       ], dtype=float32)

In [ ]:
63.62069/2

31.810345

In [ ]:
c1[0,3]

array([ 32, 142, 209], dtype=uint8)

In [ ]:
@cuda.jit
def convert_bgr2hsv_kernel(in_pixels, out_pixels):
    r_ ,c= cuda.grid(2)
    if r_ < img.shape[0] and c < img.shape[1]:
        # b,g,r = in_pixels[r_,c,0],in_pixels[r_,c,1],in_pixels[r_,c,2]
        S=0
        H=0
        V = max(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0])
        # Vsmin=V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0])
        if V!=0:
            S=(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))/V #

        if in_pixels[r_,c,2]==in_pixels[r_,c,1] and in_pixels[r_,c,1]==in_pixels[r_,c,0]:
            H=0
        elif V==in_pixels[r_,c,2]:
            H= 60*(in_pixels[r_,c,1]-in_pixels[r_,c,0])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))
        elif V==in_pixels[r_,c,1]:
            H= 120 + 60*(in_pixels[r_,c,0]-in_pixels[r_,c,2])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))
        else:
            H= 240 + 60*(in_pixels[r_,c,2]-in_pixels[r_,c,1])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))

        if H<0:
            H = H + 360

    
        out_pixels[r_,c,0] = H
        out_pixels[r_,c,1] = S
        out_pixels[r_,c,2] = V
    


# tuần tự
img=cv.imread("/content/Test_4.jpg").astype(np.float32)
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

#kernel
img=cv.imread("/content/Test_4.jpg")#.astype(np.float32)
d_m2 = cuda.device_array(img.shape, dtype=np.float32)
d_in_img = cuda.to_device(img)
convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)
c3=d_m2.copy_to_host()

# openCV
img=cv.imread("/content/Test_4.jpg").astype(np.float32)
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)
np.mean(np.abs(c3.astype(np.float32)- c2.astype(np.float32)))

0.0

In [ ]:
#kernel
img=cv.imread("/content/Test_4.jpg")#.astype(np.float32)
d_m2 = cuda.device_array(img.shape, dtype=np.float32)
d_in_img = cuda.to_device(img)

In [ ]:
%%time
# convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)

CPU times: user 1.99 ms, sys: 1 ms, total: 2.99 ms
Wall time: 3.01 ms


In [ ]:
%%time
# convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)
c3=d_m2.copy_to_host()

CPU times: user 8.63 ms, sys: 994 µs, total: 9.62 ms
Wall time: 9.49 ms


### hsv1
image = cv.cvtColor(image, cv.COLOR_BGR2HSV) 
ghép và so hist

In [ ]:
@jit
def convert_bgr2hsv(in_pixels, out_pixels):
    '''
    3d & 3d
    '''
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0]/255,in_pixels[i,j,1]/255,in_pixels[i,j,2]/255
            S=0
            H=0
            V = max(r,g,b)
            if V!=0:
                S=round((V-min(r,g,b))/V ,3)
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            if i==287 and j==536  :
              print(i,j,"H",H/2)
            out_pixels[i,j,0] = round(H/2+0.006,0)
            out_pixels[i,j,1] =  round(255*S,0)
            out_pixels[i,j,2] = 255*V
    return out_pixels

img=cv.imread("/content/Test_4.jpg")
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread("/content/Test_4.jpg")
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)

print("----------------error----------")
np.mean(np.abs(c1.astype(np.float32)- c2.astype(np.float32))) # up low

287 536 H 44.49438202247191
----------------error----------


0.021694712

In [ ]:
#2222
@jit
def convert_bgr2hsv(in_pixels, out_pixels):
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0]/255,in_pixels[i,j,1]/255,in_pixels[i,j,2]/255
            S=0
            H=0
            V = max(r,g,b)
            if V!=0:
                S=(V-min(r,g,b))/V
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            if i==287 and j==536  :
              print(i,j,"H",H/2)
            out_pixels[i,j,0] =H/2
            out_pixels[i,j,1] =  255*S
            out_pixels[i,j,2] = 255*V
    return out_pixels

img=cv.imread("/content/Test_4.jpg")
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread("/content/Test_4.jpg")
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)

print("----------------error----------")
np.mean(np.abs(c1.astype(np.float32)- c2.astype(np.float32))) # up low

287 536 H 44.49438202247191
----------------error----------


0.31183895

In [ ]:
l0=[]
l1=[]
l2=[]
t=0
k=0
for i in range(img.shape[0]):
  for j in range(img.shape[1]):
    if  c1[i,j,0]!=c2[i,j,0]:
      k=k+1
      if  c1[i,j,0]>c2[i,j,0]:
        t=t+1
      # print(i,j,c1[i,j,0],"!=",c2[i,j,0])
      if img[i,j,0] not in l0:
        l0.append(img[i,j,0])
      if img[i,j,1] not in l1:
        l1.append(img[i,j,1])
      if img[i,j,2] not in l2:
        l2.append(img[i,j,2])

In [ ]:
def in_(a):
  print("min",min(a))
  print("max",max(a))
  a.sort()
  print(a)

In [ ]:
in_(l0)

NameError: ignored

In [ ]:
len(l)

152

In [ ]:
# l.sort()
l

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 124,
 125,
 126,
 128,
 129,
 130,
 132,
 133,
 135,
 138,
 140,
 142,
 143,
 144,
 145,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179]

In [ ]:
k

1259866

In [ ]:
@cuda.jit
def convert_bgr2hsv_kernel(in_pixels, out_pixels):
    r_ ,c= cuda.grid(2)
    if r_ < img.shape[0] and c < img.shape[1]:
        # b,g,r = in_pixels[r_,c,0],in_pixels[r_,c,1],in_pixels[r_,c,2]
        S=0
        H=0
        V = max(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0])
        # Vsmin=V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0])
        if V!=0:
            S=(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))/V #

        if in_pixels[r_,c,2]==in_pixels[r_,c,1] and in_pixels[r_,c,1]==in_pixels[r_,c,0]:
            H=0
        elif V==in_pixels[r_,c,2]:
            H= 60*(in_pixels[r_,c,1]-in_pixels[r_,c,0])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))
        elif V==in_pixels[r_,c,1]:
            H= 120 + 60*(in_pixels[r_,c,0]-in_pixels[r_,c,2])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))
        else:
            H= 240 + 60*(in_pixels[r_,c,2]-in_pixels[r_,c,1])/(V-min(in_pixels[r_,c,2],in_pixels[r_,c,1],in_pixels[r_,c,0]))

        if H<0:
            H = H + 360

    
        out_pixels[r_,c,0] = H/2
        out_pixels[r_,c,1] = 255*S
        out_pixels[r_,c,2] = V
    


# tuần tự
img=cv.imread("/content/Test_4.jpg").astype(np.float32)
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

#kernel
img=cv.imread("/content/Test_4.jpg")#.astype(np.float32)
d_m2 = cuda.device_array(img.shape, dtype=np.float32)
d_in_img = cuda.to_device(img)
convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)
c3=d_m2.copy_to_host()

# openCV
img=cv.imread("/content/Test_4.jpg").astype(np.float32)
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)
np.mean(np.abs(c3.astype(np.float32)- c2.astype(np.float32)))

287 536 H 44.49438202247191


0.16317198

In [ ]:
#kernel
img=cv.imread("/content/Test_4.jpg")#.astype(np.float32)
d_m2 = cuda.device_array(img.shape, dtype=np.float32)
d_in_img = cuda.to_device(img)

In [ ]:
%%time
# convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)

CPU times: user 1.99 ms, sys: 1 ms, total: 2.99 ms
Wall time: 3.01 ms


In [ ]:
%%time
# convert_bgr2hsv_kernel[grid_size_2, BLOCK_SIZE_2](d_in_img,d_m2)
c3=d_m2.copy_to_host()

CPU times: user 8.63 ms, sys: 994 µs, total: 9.62 ms
Wall time: 9.49 ms


### ghép 1
sai số hàm bgr2HSV

In [ ]:
def fd_histogram(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    print("image input hsv ",image.dtype)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    print("image output hsv ",image.dtype)
    temp=image.copy()
    print(temp[0,0])
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins],  [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g = getFigureForImage("/content/Test_4.jpg")

image input hsv  uint8
image output hsv  uint8
[0 0 0]


In [ ]:
def fd_histogram4(image, mask=None):
    bins=8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    o_img =image.copy().astype(np.float32)
    image = convert_bgr2hsv(image,o_img)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins],  [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments4(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram4(img).astype(np.float64)
    hu_monents = fd_hu_moments4(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g2 = getFigureForImage("/content/Test_4.jpg")

287 536 H 29.71698113207547


In [ ]:
np.mean(np.abs(g.astype(np.float32)- g2.astype(np.float32))) 

0.0071124756

In [ ]:
np.mean(g)

0.005815248872517536

### ghép

In [ ]:
def convert_bgr2hsv(in_pixels, out_pixels):
    '''
    3d & 3d
    '''
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0]/255.0,in_pixels[i,j,1]/255.0,in_pixels[i,j,2]/255.0
            S=0
            H=0
            V = max(r,g,b)
            if V!=0:
                S=(V-min(r,g,b))/V 
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            out_pixels[i,j,0] = H/2
            out_pixels[i,j,1] =  255*S
            out_pixels[i,j,2] =255*V
    return out_pixels

img=cv.imread("/content/Test_4.jpg")
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread("/content/Test_4.jpg")
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)

print("----------------error----------")
np.mean(np.abs(c1.astype(np.float32)- c2.astype(np.float32))) # up low

----------------error----------


0.31183895

In [ ]:
def fd_histogram(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    print("image input hsv ",image.dtype)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    print("image output hsv ",image.dtype)
    temp=image.copy()
    print(temp[0,0])
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g = getFigureForImage("/content/Test_4.jpg")

image input hsv  uint8
image output hsv  uint8
[0 0 0]


In [ ]:
def fd_histogram4(image, mask=None):
    bins=8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    o_img =image.copy()
    image = convert_bgr2hsv(image,o_img)
    print(image)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments4(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram4(img).astype(np.float64)
    hu_monents = fd_hu_moments4(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g2 = getFigureForImage("/content/Test_4.jpg")

In [ ]:
np.mean(np.abs(g.astype(np.float32)- g2.astype(np.float32))) 

0.007101606

In [ ]:
np.mean(g)

0.005815248872517536

### ghép gốc

In [ ]:
def fd_histogram(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage(path):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)
    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    hist_figure = fd_histogram(img).astype(np.float64)
    hu_monents = fd_hu_moments(img)
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g0 = getFigureForImage("/content/Test_4.jpg")

### ghép ss hu + hist

In [ ]:
def fd_histogram4(image, mask=None):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 255, 0, 255, 0, 255])
    # normalize the histogram
    cv.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

def fd_hu_moments4(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature
  
def getFigureForImage4(path):
    img = cv.imread(path)

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)

    mask_img = applyCannyThreshold(gray, 12)
    mask_img = zipImage(mask_img, 8, 8, 0.12)
    mask_img = zipImage(mask_img, 16, 16, 0.2)
    mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)
    mask_img = joinNeiboorPixel(mask_img, 16, 16, 3, 1 / 3)
    for chanel in range(0, 3):
        img[:,:,chanel] = img[:,:,chanel] * mask_img

    # hu_monents = fd_hu_moments4(img)
    
    hist_figure = fd_histogram4(img).astype(np.float64)
    
    fig = np.concatenate((hist_figure, hu_monents))
    return fig
g4 = getFigureForImage4("/content/Test_4.jpg")

In [ ]:
np.mean(np.abs(g.astype(np.float32)- g2.astype(np.float32))) 

0.007101606

### nháp

In [ ]:
#252

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 4. Nhìn lại quá trình làm đồ án

**Khó khăn**: 
+ Quý: 
+ Tài: 
+ Mận: 


**Học được:**
+ Quý: 
+ Tài: 
+ Mận: 


**Nếu có thêm thời gian**: nhóm sẽ nghiên cứu sâu hơn về ...


---

# 5. Tài liệu tham khảo

- Nhập môn học máy: [BÀI TẬP THỰC HÀNH II ](https://drive.google.com/file/d/1U65IfdkizLD0pd1aQKp9xqVRnTj5bejR/view)
- [Image moment - Wiki](https://en.wikipedia.org/wiki/Image_moment)

- https://towardsdatascience.com/using-nmf-to-classify-companies-a77e176f276f
- https://github.com/jonasrothfuss/fishervector
- https://hal.inria.fr/hal-00779493/file/RR-8209.pdf
- https://viblo.asia/p/sift-scale-invariant-feature-transform-huan-luyen-mo-hinh-cho-cac-bai-toan-phan-loai-924lJqJaZPM
- https://xgboost.readthedocs.io/en/stable/
- https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663
- https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb
- https://www.tensorflow.org/tutorials/load_data/images#using_tfdata_for_finer_control
- https://www.smartdraw.com/flowchart/flowchart-symbols.htm
- https://en.wikipedia.org/wiki/Image_moment#cite_note-%E2%80%9Chu-1
- [OpenCV RGB histogram calculation and drawing----calcHist() function, normalize() function](https://www.programmerall.com/article/5123272509/)

